In [46]:
import pandas as pd
import shapefile, csv, math
from shapely.geometry import Point
import scipy.spatial
import numpy as np

import pygplates
from parameters import parameters as param

# input: degrees between two points on sphere
# output: straight distance between the two points (assume the earth radius is 1)
# to get the kilometers, use the return value to multiply by the real earth radius
def degree_to_straight_distance(degree):
    return math.sin(math.radians(degree)) / math.sin(math.radians(90 - degree/2.))

data=pd.read_csv("EarthChem_all.csv", sep=',', skipinitialspace=True)
c='ND'
#print(data.columns.values)
a=data.columns.get_loc(c)
df=data.iloc[:,[6,7,10,a]] 
#6,7,10 are the default column numbers for Long, Lat and Age. Please check their position beforehand 
df=df[(df!=0).all(1)]
df=df.dropna()
#print(len(df))

# build the tree of the trench points
t=0
trench_points_filename = param['convergence_data_dir'] + param['convergence_data_filename_prefix'] +\
    '_{:0.2f}'.format(t) + "." + param['convergence_data_filename_ext']
data=np.loadtxt(trench_points_filename)
print(trench_points_filename)

points_3d = [pygplates.PointOnSphere((row[1],row[0])).to_xyz() for row in data]
points_tree = scipy.spatial.cKDTree(points_3d)

data_index=[]
index_count=-1
candidates=[]
for index, row in df.iterrows():
    index_count+=1
    try:
        candidates.append(pygplates.PointOnSphere((row[0], row[1])).to_xyz())#lat, lon
        data_index.append(index_count)
    except pygplates.InvalidLatLonError:
        print('invalid lat or lon: ',row)
        continue

region = 1 #degrees
dists, indices = points_tree.query(
            candidates, k=1, distance_upper_bound=degree_to_straight_distance(region))

result_index = np.array(data_index)[indices<len(points_3d)]
print(len(result_index))

# create a point shapefile
sf = shapefile.Writer(shapefile.POINT)
# for every record there must be a corresponding geometry.
sf.autoBalance = 1

# create the field names and data type for each.
sf.field("AGE", "N")
sf.field(c, "C")
#"C": Characters, text.
#"N": Numbers, with or without decimals.
#"F": Floats (same as "N").
#"L": Logical, for boolean True/False values.
#"D": Dates.
#"M": Memo, has no meaning within a GIS and is part of the xbase spec instead.

for idx in result_index:
    # create the point geometry
    sf.point(df.iloc[idx][1],df.iloc[idx][0]) #lon lat
    print(df.iloc[idx][1],df.iloc[idx][0])
    # add attribute data
    sf.record(df.iloc[idx][2], df.iloc[idx][3])

sf.save('out.shp')

df.iloc[result_index].to_csv("out.csv",index=False,float_format='%.4f')


./convergence_data/subStats_0.00.csv
('invalid lat or lon: ', LATITUDE       49.24
LONGITUDE   -1295.00
AGE            44.25
ND             12.60
Name: 23956, dtype: float64)
('invalid lat or lon: ', LATITUDE     110.7560
LONGITUDE    -38.0775
AGE           27.0000
ND            30.9000
Name: 33160, dtype: float64)
('invalid lat or lon: ', LATITUDE     110.8040
LONGITUDE    -38.0939
AGE           27.0000
ND            22.1000
Name: 33165, dtype: float64)
('invalid lat or lon: ', LATITUDE     110.8040
LONGITUDE    -38.0939
AGE           27.0000
ND            21.3000
Name: 33171, dtype: float64)
('invalid lat or lon: ', LATITUDE    -106.668
LONGITUDE    -37.510
AGE           28.500
ND            31.000
Name: 41450, dtype: float64)
('invalid lat or lon: ', LATITUDE    -107.003
LONGITUDE    -37.478
AGE           28.500
ND            30.000
Name: 41451, dtype: float64)
('invalid lat or lon: ', LATITUDE    -107.003
LONGITUDE    -37.478
AGE           28.500
ND            30.000
Name: 41454, d

(133.17, 29.75)
(138.5, 34.5)
(138.5, 34.5)
(138.5, 34.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143.0, 38.5)
(143

(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.03, 40.78)
(14.2, 40.9)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.0333, 41.0569)
(14.0333, 41.0569)
(14.0333, 41.0569)
(14.0333, 41.0569)
(14.0333, 41.0569)
(14.0333, 41.0569)
(14.0333, 41.0569)
(13.95, 41.1319)
(13.95, 41.1319)
(14.0875, 40.9667)
(14.0875, 40.9667)
(13.9789, 40.8031)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78

(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(-78.32, 3.0)
(120.35, 15.13)
(120.35, 15.13)
(120.35, 15.13)
(120.35, 15.13)
(152.5, 46.5)
(26.9598, 36.7378)
(26.9573, 36.739)
(26.9523, 36.6945)
(26.9507, 36.699)
(26.9477, 36.7033)
(26.9717, 36.7228)
(26.9558, 36.724)
(26.9567, 36.7257)
(26.9567, 36.7257)
(26.9893, 36.7602)
(26.9893, 36.7602)
(27.0793, 36.7713)
(27.0813, 36.7692)
(26.963, 36.7117)
(26.9637, 36.711)
(26.9652, 36.678)
(27.0195, 36.7687)
(-90.9328, 12.7168)
(-90.9328, 12.7168)
(-90.9328, 12.7168)
(-90.9328, 12.7168)
(-90.9328, 12.7168)
(-90.9447, 12.6705)
(-90.9437, 12.6825)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12.7165)
(-90.932, 12

(169.815, -43.2319)
(179.22799999999995, -39.5608)
(-178.42, -30.7)
(-178.42, -30.7)
(-178.42, -30.7)
(-178.42, -30.7)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(12.65, 41.73)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(12.65, 41.73)
(12.65, 41.73)
(11.98, 42.63)
(11.98, 42.63)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(13.9, 40.7)
(39.0, 37.0)
(39.0, 37.0)
(39

(14.3735, 40.7232)
(23.0, 48.3)
(23.0, 48.3)
(22.7, 48.65)
(22.7, 48.65)
(22.35, 48.78)
(22.7, 48.65)
(22.35, 48.78)
(22.7, 48.65)
(22.7, 48.65)
(22.35, 48.78)
(23.2, 47.9)
(22.7, 48.65)
(22.7, 48.65)
(23.0, 48.0)
(22.35, 48.78)
(23.2, 47.9)
(22.8, 48.0)
(22.8, 48.0)
(22.75, 48.0)
(22.8, 48.0)
(22.8, 48.0)
(23.2, 47.9)
(22.75, 48.0)
(22.75, 47.95)
(22.75, 47.95)
(22.75, 47.95)
(22.75, 47.95)
(22.3283, 42.67)
(21.8711, 43.6433)
(21.8711, 43.6433)
(21.8711, 43.6433)
(21.8711, 43.6433)
(21.8711, 43.6433)
(17.0, 47.2)
(16.98, 49.2)
(12.47, 41.85)
(12.47, 41.85)
(12.25, 42.05)
(12.25, 42.05)
(12.25, 42.05)
(12.25, 42.05)
(12.25, 42.05)
(12.25, 42.05)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.25, 42.05)
(12.25, 42.05)
(12.25, 42.05)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(12.47, 41.85)
(

(8.75, 47.77)
(-173.92, -15.87)
(-173.92, -15.87)
(-177.87, -29.25)
(-177.87, -29.25)
(-177.87, -29.25)
(-177.87, -29.25)
(-177.87, -29.25)
(-177.87, -29.25)
(-177.87, -29.25)
(155.75, -7.08)
(156.58, -7.75)
(156.58, -7.75)
(156.58, -7.75)
(156.58, -7.75)
(156.58, -7.75)
(157.8, -8.38)
(156.97, -7.89)
(156.97, -7.89)
(142.3, 27.3)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(139.02, 35.22)
(11.83, 42.63)
(11.83, 42.63)
(11.83, 42.63)
(10.17, 42.77)
(10.17, 42.77)
(10.17, 42.77)
(12.65, 41.73)
(16.33, 47.6)
(16.33, 47.6)
(24.3, 38.7)
(24.57, 38.92)
(24.57, 38.92)
(14.2, 40.9)
(14.2, 40.9)
(14.2, 40.9)
(139.42, 34.75)
(138.5, 34.5)
(139.29, 34.4)
(13.5967, 40.8494)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.05, 40.78)
(14.03, 40.78)
(-86.0, 10.0)
(-84.5, 9.5)
(-81.0437, 6.122999999999998)
(13.95, 41.1319)
(13.95, 41.1319)
(13.95, 41.131

(138.29, 9.50333)
(138.29, 9.50333)
(137.25, 7.77333)
(10.0767, 44.0733)
(-143.665, 61.2611)
(-144.08, 61.3625)
(-144.27700000000004, 61.4333)
(-153.327, 57.1222)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5083, 40.6583)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.5, 40.65)
(14.2167, 40.65)
(14.2167, 40.65)
(14.2167, 40.65)
(14.2167, 40.65)
(14.2167, 

In [35]:
print(len(indices))
print(len(points_3d))
print(len(np.array(candidates)[indices<len(points_3d)]))

135721
2970
4022


In [48]:
df.iloc[result_index].to_csv("out.csv",index=False,float_format='%.3f')